## EKF
使用EKF进行移动机器人的位姿预测，见相关[博客](https://blog.csdn.net/weixin_42301220/article/details/124605350?csdn_share_tail=%7B%22type%22%3A%22blog%22%2C%22rType%22%3A%22article%22%2C%22rId%22%3A%22124605350%22%2C%22source%22%3A%22weixin_42301220%22%7D&ctrtid=Diyk2)

In [116]:
import PyQt5


In [117]:
import math
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib qt5

# # set up matplotlib
# is_ipython = 'inline' in matplotlib.get_backend()
# if is_ipython:
#     from IPython import display

plt.ion()


import numpy as np
from scipy.spatial.transform import Rotation as Rot

设置过程噪声和观测噪声的协方差矩阵

In [118]:
Q = np.matrix([[0.1, 0, 0, 0],  # variance of location on x-axis
        [0, 0.1, 0, 0],  # variance of location on y-axis
        [0, 0, np.deg2rad(1.0), 0],  # variance of yaw angle
        [0, 0, 0, 1.0]])**2  # variance of velocity

R = np.matrix([[1.0,0.0],[0.0,1.0]])**2

仿真参数

In [119]:
#  Simulation parameter
INPUT_NOISE = np.diag([1.0, np.deg2rad(30.0)]) ** 2
GPS_NOISE = np.diag([0.5, 0.5]) ** 2

DT = 0.1  # time tick [s]
SIM_TIME = 50.0  # simulation time [s]

show_animation = True



状态向量

x_t = [x,y,yaw,v]

控制输入为

u = [v,w]

控制输入表示

In [120]:
def control_input():
    v = 1.0  # 线速度，m/s
    yaw_rate = 0.1  #角速度，rad/s
    u = np.matrix([
            [v],
            [yaw_rate]
            ])
    return u

观测模型

In [121]:
def observation_model(x):
    H = np.matrix([[1.0,0,0,0],[0,1.0,0,0]])
    z = H@x
    return z


过程模型（状态方程）

In [122]:
def motion_model(x,u):
    F = np.matrix([
            [1.0,0,0,0],
            [0,1.0,0,0],
            [0,0,1.0,0],
            [0,0,0,0]
            ])
    B = np.matrix([
            [DT*np.cos(x[2,0]),0],
            [DT*np.sin(x[2,0]),0],
            [0,DT],
            [1.0,0]
            ])
    x = F@x+B@u
    return x
    

观测向量获取

In [123]:
def observation(x_true, x_d,u):
    x_true = motion_model(x_true,u)
    # add noise to gps x-y
    z = observation_model(x_true) + GPS_NOISE @ np.random.randn(2, 1)

    # add noise to input
    ud = u + INPUT_NOISE @ np.random.randn(2, 1)

    xd = motion_model(x_d, ud)

    return x_true, z, xd, ud


F的雅可比矩阵定义

In [124]:
def jacob_f(x,u):
    """
    Jacobian of Motion Model

    motion model
    x_{t+1} = x_t+v*dt*cos(yaw)
    y_{t+1} = y_t+v*dt*sin(yaw)
    yaw_{t+1} = yaw_t+omega*dt
    v_{t+1} = v{t}
    so
    dx/dyaw = -v*dt*sin(yaw)
    dx/dv = dt*cos(yaw)
    dy/dyaw = v*dt*cos(yaw)
    dy/dv = dt*sin(yaw)
    """
    yaw = x[2,0]
    v = u[0,0]
    J_F = np.matrix([
            [1.0,0.0,-v*np.sin(yaw)*DT,np.cos(yaw)*DT],
            [0.0,1.0,v*np.cos(yaw)*DT,np.sin(yaw)*DT],
            [0,0,1,0],
            [0,0,0,1]
            ])
    return J_F


H的雅可比矩阵定义

In [125]:
def jacob_h():
    """
    Jacobian of Observation Model
    """
    J_H = np.matrix([[1.0,0,0,0],[0,1.0,0,0]])
    return J_H

ekf估计

In [126]:
def ekf_estimation(x_esti,P_esti,z,u):
    """两个阶段，预测和更新

    Args:
        x_esti (_type_): 估计的状态
        P_esti (_type_): 估计的P矩阵（后验估计误差协方差矩阵）
        z (_type_): 观测向量
        u (_type_): 控制输入
    """
    #  Predict
    x_pred = motion_model(x_esti,u)
    J_F = jacob_f(x_esti,u)
    P_pred = J_F@P_esti@J_F.T+Q

    # update
    J_H = jacob_h()
    z_pred = observation_model(x_pred)
    y = z-z_pred
    S = J_H@P_pred@J_H.T+R
    K = P_pred@J_H.T@np.linalg.pinv(S)
    x_esti = x_pred+K@y
    P_esti = (np.eye(len(x_esti))-K@J_H)@P_pred
    return x_esti,P_esti

画图

In [127]:
def plot_covariance_ellipse(x_esti, P_esti):  # pragma: no cover
    Pxy = P_esti[0:2, 0:2]
    # 计算方形矩阵的特征值和特征向量
    eigval, eigvec = np.linalg.eig(Pxy)

    if eigval[0] >= eigval[1]:
        bigind = 0
        smallind = 1
    else:
        bigind = 1
        smallind = 0

    t = np.arange(0, 2 * math.pi + 0.1, 0.1)
    a = math.sqrt(eigval[bigind])
    b = math.sqrt(eigval[smallind])
    x = [a * math.cos(it) for it in t]
    y = [b * math.sin(it) for it in t]
    angle = math.atan2(eigvec[1, bigind], eigvec[0, bigind])
    rot = Rot.from_euler('z', angle).as_matrix()[0:2, 0:2]
    fx = rot @ (np.array([x, y]))
    px = np.array(fx[0, :] + x_esti[0, 0]).flatten()
    py = np.array(fx[1, :] + x_esti[1, 0]).flatten()
    plt.plot(px, py, "--r")


主函数入口

In [128]:
def main():
    time=0.0
    # State Vector [x y yaw v]'
    x_esti = np.zeros((4,1))
    x_true = np.zeros((4,1))
    P_esti = np.eye(4)
    # Dead reckoning
    x_dr = np.zeros((4,1))

    # history
    h_x_esti = x_esti
    h_x_true = x_true
    h_x_dr = x_dr
    h_z = np.zeros((2, 1))


    while SIM_TIME>=time:
        time += DT
        u = control_input()
        x_true, z, x_dr, ud = observation(x_true, x_dr, u)
        
        x_esti, P_esti = ekf_estimation(x_esti,P_esti,z,ud)

        # store data history
        h_x_esti = np.hstack((h_x_esti, x_esti))
        h_x_dr = np.hstack((h_x_dr, x_dr))
        h_x_true = np.hstack((h_x_true, x_true))
        h_z = np.hstack((h_z, z))

        if show_animation:
            plt.cla()
            # for stopping simulation with the esc key.
            plt.gcf().canvas.mpl_connect('key_release_event',
                lambda event: [exit(0) if event.key == 'escape' else None])
            plt.plot(h_z[0, :], h_z[1, :], ".g")
            plt.plot(h_x_true[0, :].flatten(),
                    h_x_true[1, :].flatten(), "-b")
            plt.plot(h_x_dr[0, :].flatten(),
                    h_x_dr[1, :].flatten(), "-k")
            plt.plot(h_x_esti[0, :].flatten(),
                    h_x_esti[1, :].flatten(), "-r")
            plot_covariance_ellipse(x_esti, P_esti)
            plt.axis("equal")
            plt.grid(True)
            plt.pause(0.001)


In [129]:
main()

KeyboardInterrupt: 